<a href="https://colab.research.google.com/github/AikaMinami/uas-big-data-2022/blob/main/FinalProjectBigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Java**

In [83]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# **Install Apache Spark**

In [84]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# **Install PySpark**

In [ ]:
!pip install -q findspark

In [85]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [86]:
import findspark
findspark.init()

In [87]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("BDFP")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

In [88]:
!wget --continue https://data.nasa.gov/resource/2vr3-k9wn.json -O /tmp/comets.json

--2022-06-29 06:42:33--  https://data.nasa.gov/resource/2vr3-k9wn.json
Resolving data.nasa.gov (data.nasa.gov)... 128.102.186.77, 2001:4d0:6311:2c05:60b0:5ad8:1210:ea07
Connecting to data.nasa.gov (data.nasa.gov)|128.102.186.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘/tmp/comets.json’

/tmp/comets.json        [ <=>                ]  40.61K  --.-KB/s    in 0.1s    

2022-06-29 06:42:34 (315 KB/s) - ‘/tmp/comets.json’ saved [41585]



In [89]:
df = spark.read.json("/tmp/comets.json", multiLine=True)

In [90]:
df.printSchema()

root
 |-- designation: string (nullable = true)
 |-- discovery_date: string (nullable = true)
 |-- h_mag: string (nullable = true)
 |-- i_deg: string (nullable = true)
 |-- moid_au: string (nullable = true)
 |-- orbit_class: string (nullable = true)
 |-- period_yr: string (nullable = true)
 |-- pha: string (nullable = true)
 |-- q_au_1: string (nullable = true)
 |-- q_au_2: string (nullable = true)



In [91]:
df.show(4,False)

+-------------------+-----------------------+-----+-----+-------+-----------+---------+---+------+------+
|designation        |discovery_date         |h_mag|i_deg|moid_au|orbit_class|period_yr|pha|q_au_1|q_au_2|
+-------------------+-----------------------+-----+-----+-------+-----------+---------+---+------+------+
|419880 (2011 AH37) |2011-01-07T00:00:00.000|19.7 |9.65 |0.035  |Apollo     |4.06     |Y  |0.84  |4.26  |
|419624 (2010 SO16) |2010-09-17T00:00:00.000|20.5 |14.52|0.028  |Apollo     |1        |Y  |0.93  |1.08  |
|414772 (2010 OC103)|2010-07-28T00:00:00.000|19   |23.11|0.333  |Apollo     |1.31     |N  |0.39  |2     |
|414746 (2010 EH20) |2010-03-06T00:00:00.000|18   |23.89|0.268  |Amor       |4.24     |N  |1.25  |3.99  |
+-------------------+-----------------------+-----+-----+-------+-----------+---------+---+------+------+
only showing top 4 rows



In [92]:
df.count()

202

In [93]:
df.select("designation", "discovery_date", "orbit_class").show(5)

+-------------------+--------------------+-----------+
|        designation|      discovery_date|orbit_class|
+-------------------+--------------------+-----------+
| 419880 (2011 AH37)|2011-01-07T00:00:...|     Apollo|
| 419624 (2010 SO16)|2010-09-17T00:00:...|     Apollo|
|414772 (2010 OC103)|2010-07-28T00:00:...|     Apollo|
| 414746 (2010 EH20)|2010-03-06T00:00:...|       Amor|
|407324 (2010 OB101)|2010-07-18T00:00:...|     Apollo|
+-------------------+--------------------+-----------+
only showing top 5 rows



In [94]:
print("Is it Potentially Hazardous Asteroids?")
df.groupBy('pha').count().sort(df.pha).show()

Is it Potentially Hazardous Asteroids?
+---+-----+
|pha|count|
+---+-----+
|  N|  151|
|  Y|   30|
|n/a|   21|
+---+-----+



In [95]:
df_filtered = df.filter("pha == 'N' AND h_mag IS NOT NULL")
df_filtered.show(50, False)

+-------------------+-----------------------+-----+-----+-------+-----------+---------+---+------+------+
|designation        |discovery_date         |h_mag|i_deg|moid_au|orbit_class|period_yr|pha|q_au_1|q_au_2|
+-------------------+-----------------------+-----+-----+-------+-----------+---------+---+------+------+
|414772 (2010 OC103)|2010-07-28T00:00:00.000|19   |23.11|0.333  |Apollo     |1.31     |N  |0.39  |2     |
|414746 (2010 EH20) |2010-03-06T00:00:00.000|18   |23.89|0.268  |Amor       |4.24     |N  |1.25  |3.99  |
|407324 (2010 OB101)|2010-07-18T00:00:00.000|20.7 |9.12 |0.111  |Apollo     |2.06     |N  |0.77  |2.46  |
|381989 (2010 HR80) |2010-04-28T00:00:00.000|19.9 |26.71|0.104  |Apollo     |1.56     |N  |0.68  |2.02  |
|369454 (2010 NZ1)  |2010-07-09T00:00:00.000|19.4 |32.78|0.275  |Apollo     |1.61     |N  |0.49  |2.26  |
|365449 (2010 NJ1)  |2010-07-03T00:00:00.000|20.3 |11.23|0.155  |Aten       |0.95     |N  |0.44  |1.49  |
|356394 (2010 QD2)  |2010-08-21T00:00:00.000|1

In [103]:
from pyspark.sql.functions import max

maxValue = df_filtered.agg(max("period_yr")).collect()[0][0]
print("max Period Years: ",maxValue)
df_filtered.select("designation","period_yr").filter(df_filtered.period_yr == maxValue).show(20, False)

max Period Years:  99.82
+-----------+---------+
|designation|period_yr|
+-----------+---------+
|(2014 PP69)|99.82    |
+-----------+---------+



In [104]:
from pyspark.sql.functions import min

minValue = df_filtered.agg(min("period_yr")).collect()[0][0]
print("min Period Years: ",minValue)
df_filtered.select("designation","period_yr").filter(df.period_yr== minValue).show(20, False)

min Period Years:  0.72
+------------+---------+
|designation |period_yr|
+------------+---------+
|(2010 HX107)|0.72     |
+------------+---------+



In [106]:
df_filtered.groupBy('h_mag').count().sort(df_filtered.h_mag).show()

+-----+-----+
|h_mag|count|
+-----+-----+
| 15.6|    1|
| 16.2|    1|
|   17|    1|
| 17.2|    3|
| 17.3|    1|
| 17.4|    1|
| 17.5|    1|
| 17.7|    1|
|   18|    2|
| 18.1|    1|
| 18.2|    2|
| 18.3|    3|
| 18.5|    1|
| 18.7|    3|
| 18.8|    3|
|   19|    2|
| 19.1|    3|
| 19.2|    5|
| 19.3|    3|
| 19.4|    5|
+-----+-----+
only showing top 20 rows



In [99]:
df.groupBy('orbit_class').agg(max("moid_au")).sort(df.orbit_class).show()

+--------------------+------------+
|         orbit_class|max(moid_au)|
+--------------------+------------+
|                Amor|       1.617|
|              Apollo|       0.703|
|                Aten|       0.316|
|               Comet|       6.373|
|    Encke-type Comet|        0.63|
|  Halley-type Comet*|       0.828|
|Jupiter-family Comet|       1.683|
|Jupiter-family Co...|       2.945|
|     Parabolic Comet|       1.546|
+--------------------+------------+

